# Folium



> folium makes it easy to visualize data that’s been manipulated in Python on an interactive leaflet map. It enables both the binding of data to a map for choropleth visualizations as well as passing rich vector/raster/HTML visualizations as markers on the map.
From [folium homepage](https://python-visualization.github.io/folium/)

Folium is a python module that makes easy to produce maps, either static or with some light interactivity (zoom, panning, tooltips, ...)

It leverages existing database of geographical data like [OpenStreetMap](https://www.openstreetmap.org) and [Stamen](maps.stamen.com/), and it's built on the [leaflet.js](https://leafletjs.com/) library for the basic map functionality. It's a bit like Seaborn being built on matplotlib, but for maps.

If you are more interested in producing static images you may want to check out an alternative to Folium called [Cartopy](https://scitools.org.uk/cartopy/docs/latest/).



# Basic maps with Folium

Let's take a look at the default map

In [71]:
import folium
folium.__version__

'0.12.1.post1'

In [72]:
map = folium.Map()
map

Setting initial location and zoom level

In [73]:
map = folium.Map(
    location = [45, 0],
    zoom_start = 4
)
map

Changing the tile set, see the [documentation](https://python-visualization.github.io/folium/modules.html) for built-in tiles.

In [74]:
map = folium.Map(
    location = [45, 0],
    zoom_start = 4,
    tiles = 'Stamen Toner'

    #default would be
    #tiles = 'OpenStreetMap'
)
map

In [75]:
map = folium.Map(
    location = [45, 0],
    zoom_start = 8,
    tiles = 'Stamen Terrain'
)
map

Adding a minimap



In [76]:
from folium import plugins

minimap = plugins.MiniMap()

map.add_child(minimap)
map

Saving the map to html (not many options here).

In [ ]:
map.save('mymap.html')

---

# ASSIGNMENT! Remove interactivity

Create a map with a starting location and zoom of your choice, and find the options to make the map completely static: no dragging, zooming with the button or with the mouse scrolling.

---


In [ ]:
# your solution here

# Markers

All the stuff that you want to add to a map must stay inside a `FeatureGroup` object ([doc](https://python-visualization.github.io/folium/modules.html#folium.map.FeatureGroup)). As the name implies, a `FeatureGroup` can contain many things (hence it's called group). 

In the following example we add two `CircleMarker`s to the map ([doc](https://python-visualization.github.io/folium/modules.html#folium.vector_layers.CircleMarker))

In [ ]:
#declaring the map, we are going to France
west_france = folium.Map(
    location = [45, 0],
    zoom_start = 10
)

#the FeatureGroup that will contain all the markers
libourne = folium.map.FeatureGroup()

#declaring the first marker
m = folium.features.CircleMarker(
        [44.90, -0.25],
        fill=True,
        popup="Libourne city center",
    )


#adding the marker to the group
libourne.add_child(m)

#adding second marker in a single instruction
libourne.add_child(folium.features.CircleMarker(
        [44.85, -0.58],
        fill=True,
        popup="Bourdeaux city center"
    )
)

#adding the feature group to the map
west_france.add_child(libourne)

#showing the map
west_france

---

# ASSIGNMENT! Custom markers

Update the markers in the map above doing the following:

* the marker for Bordeux is bigger
* the marker for Libourne has a red border and is filled with another color of your choice
* the marker for Bordeux is a pentagon
---

In [ ]:
# your solution here

# The usual Canada dataset

In [ ]:
import numpy as np
import pandas as pd

df_canada = pd.read_excel(
    'https://github.com/ne1s0n/dataviz_python/raw/main/resources/Canada.xlsx',
    sheet_name = 'Canada by Citizenship',
    skiprows = range(20),
    skipfooter = 2
)

#the country is in the "OdName" column, let's make it more explicit
df_canada.rename(columns = {'OdName':'Country'}, inplace = True)

#adding a "Total" column with the total immigrants for that country
years = df_canada.columns[9:43]
df_canada['Total'] = df_canada.loc[:, years].sum(axis=1)

#taking a look
df_canada.head()

# Choropleth map

A choropleth map is a map that color-codes areas of the map (e.g. countries) to express numerical values.

To do so we need two components:

* the numerical values to be plotted (easy)
* the shape of the areas on the map, usually in json forma (difficult)

For our example we have the shapes of world countries saved at the following URL:



In [ ]:
COUNTRIES_URL = 'https://github.com/ne1s0n/dataviz_python/raw/main/resources/world_countries.json'

Creating your own geojson file may be complicated. You may find geometry files online (maybe in a different format, and in that case you may need [a converter](https://products.aspose.app/gis/conversion/convert-to-geojson)).
Otherwise you may go and create your own shapes using online tools like 
[geojson.io](http://geojson.io/)

In [ ]:
#this is needed for manipulating json files
!pip install js

In [ ]:
# download countries geojson file
import json
from os.path import exists
import urllib.request 

if not exists('world_countries.json'):
  #downloading file
  urllib.request.urlretrieve(COUNTRIES_URL, "world_countries.json")

#data = io.BytesIO((await resp.arrayBuffer()).to_py())
f = open('world_countries.json',)
world_geo = json.load(f)
f.close()

print('GeoJSON file loaded!')

Just taking a look at the geojson content using adding a [`folium.GeoJson`](https://python-visualization.github.io/folium/modules.html#folium.features.GeoJson) object to a map:

In [ ]:
m = folium.Map()
m.add_child(folium.GeoJson(world_geo))
m

We can avoid explicitly loading the file and just use the URL.

In [ ]:
m = folium.Map()
m.add_child(folium.GeoJson(COUNTRIES_URL))

#show the map
m

`GeoJson` object provide options for changing aesthetics through the use of a [lambda function](https://www.w3schools.com/python/python_lambda.asp), and in particular through `lambda feature: {}` that sets the style of each feature.

Options come from `leaflet.js`, the visualization library used as a basis for building folium. Possible options, see:

* for `Point` and `MultiPoint`, see http://leafletjs.com/reference.html#marker
* for other features, see http://leafletjs.com/reference.html#path-options and http://leafletjs.com/reference.html#polyline-options


In [ ]:
m = folium.Map()
m.add_child(
folium.GeoJson(
    COUNTRIES_URL,
    style_function=lambda feature: {
        "fillColor": "#ffff00",
        "color": "black",
        "weight": 2,
        "dashArray": "5, 5",
    }
  )    
)

#show the map
m

There's a lot of potential for adaptive customization. In the following example we are using a [defaultdict](https://docs.python.org/3/library/collections.html#collections.defaultdict) object to define a default color for all countries, and the define a couple of custom colors for two countries.

In [ ]:
from collections import defaultdict

#a function that returns the default color
def default_color():
  return('yellow')

#defining the defaultdict object with a default value and
#two custom values
mycolors = defaultdict(default_color)
mycolors['Italy'] = 'red'
mycolors['China'] = 'blue'

#test that's working
mycolors['Russia']

In [ ]:
#map + custom color
m = folium.Map()
m.add_child(
folium.GeoJson(
    COUNTRIES_URL,
    style_function=lambda feature: {
        "fillColor" : mycolors[feature["properties"]["name"]],
        "color": "black",
        "weight": 2,
        "dashArray": "5, 5",
    }
  )    
)

#show the map
m

Or even more convoluted/adaptive logic (here we are using the [inline if/else ternary operator](https://docs.python.org/3/reference/expressions.html#conditional-expressions)):

In [ ]:
m = folium.Map()
m.add_child(
folium.GeoJson(
    COUNTRIES_URL,
    style_function=lambda feature: {
        "fillColor": "green"  if "e" in feature["properties"]["name"].lower() else "blue",
        "color": "black",
        "weight": 2,
        "dashArray": "5, 5",
    }
  )    
)

#show the map
m

This start to look like a choropleth, and with some additional logic we may end up creating one. That said, it's easier to directly use the [`choropleth class`](https://python-visualization.github.io/folium/modules.html#folium.features.Choropleth), that's built exactly for this:

In [ ]:
world_map = folium.Map()

world_map.add_child(
    folium.Choropleth(
      geo_data = COUNTRIES_URL,
      data = df_canada,
      columns = ['Country', 'Total'], #column 1 as the key, and column 2 the values
      key_on = 'feature.properties.name'
  )
)

# display map
world_map

A bit of aesthetics:

In [ ]:
world_map = folium.Map()

world_map.add_child(
    folium.Choropleth(
      geo_data = COUNTRIES_URL,
      data = df_canada,
      columns = ['Country', 'Total'], #column 1 as the key, and column 2 the values
      key_on = 'feature.properties.name',
      fill_color='YlOrRd', 
      fill_opacity=0.7, 
      line_opacity=0.2
  )
)

---

# ASSIGNMENT customize Choropleth

Take a look at the Choropleth documentation and improve the above map:

* add a title to the legend
* activate the highlight functionality so that when you move your mouse over a country it is highlighted
* change the number of bins so that only three colors are used
* assign a custom color to countries with missing data (e.g. Antarctica)

---

In [ ]:
#your solution here

# Not included

* [marker clusters](https://deparkes.co.uk/2016/06/24/folium-marker-clusters/)
* [heatmaps](https://blog.jovian.ai/interesting-heatmaps-using-python-folium-ee41b118a996)